In [1]:
import pandas as pd
import numpy as np
from lxml import etree as ET
from lxml.etree import XMLSyntaxError
from pymongo import MongoClient
import os
import codecs
import re

In [2]:
diretorio = "data/licitacao/"
files = os.listdir(diretorio)
files = list(map(lambda f : diretorio + f, files))

len(files)

195

In [3]:
client = MongoClient('localhost',27017)
db = client['dbOpenviz']

def save_mongo(dataframe, collection_name=None):
    result = db[collection_name].insert_many(dataframe.to_dict(orient='records'))
    return result

In [4]:
def readXMLAsString(filename):
    regex = u'[!@#$%¨&*()¬§]' # caracteres especiais a serem eliminados do arquivo
    try:
        xmlfile = codecs.open(filename,mode='r',encoding='utf-16')
        content = xmlfile.read()
        content = re.sub(regex, '',content)
        root = ET.fromstring(content)
        return root
    except Exception as ERROR:
        raise Exception(ERROR,'erro em fromXMLAsStringToDataFrame')
    finally :
        xmlfile.close()
    
def readXML(filename):
    try:
        tree = ET.parse(filename)
        root = tree.getroot()
        return root
    except XMLSyntaxError as ERROR:
        print(ERROR)
        print('XMLSyntaxError occurred  -> using fromXMLAsStringToDataFrame function to process file: ',filename)
        return readXMLAsString(filename)

def processRootElement(root):
    if root is None:
        raise ValueError("Parâmetro root não pode ser nulo")
    data = []
    for child in root:
        data.append(dict(child.attrib))
    return data

In [5]:
def determine_file_type(filename):
    index = filename.rfind('_')
    if index < 0 :
        return 'indeterminado'
    else :
        index += 1
        file_type = filename[index:]
        return file_type

In [6]:
def stripString(item):
    if type(item) == type('string'):
        return item.strip()
    else :
        return item
    
def stripStringValues(df):
    df[['nmEntidade', 'nmMunicipio']] = df[['nmEntidade', 'nmMunicipio']].applymap(stripString)
    return df

In [7]:
def getDataFrameFromLicitacao(data):
    col = ['DataReferencia', 'cdIBGE', 'dsAvaliacaoLicitacao',
       'dsClassificacaoObjetoLicitacao', 'dsClausulaProrrogacao',
       'dsModalidadeLicitacao', 'dsNaturezaLicitacao', 'dsObjeto',
       'dsRegimeExecucaoLicitacao', 'dtAbertura', 'dtEdital', 'dtEnvio',
       'idLicitacao', 'idPessoa', 'nmEntidade', 'nmMunicipio',
       'nrAnoLicitacao', 'nrEditalOrigem', 'nrLicitacao', 'nranoEditalOrigem',
       'ultimoEnvioSIMAMNesteExercicio', 'vlLicitacao']
    return pd.DataFrame(data, columns=col)

def castingDataFrameLicitacao(df_licitacao):
    casting = {
        'vlLicitacao' : 'float',
        'dtAbertura' : 'datetime64[ns]',
        'dtEdital' : 'datetime64[ns]',
        'dtEnvio' : 'datetime64[ns]',
    }
    df = df_licitacao.astype(casting,copy=False)
    return df

def processDataFrameLicitacao(data):
    df = getDataFrameFromLicitacao(data)
    df.rename(columns={'idPessoa': 'idEntidade'}, inplace=True)
    df = stripStringValues(df)
    df = castingDataFrameLicitacao(df)
    return df

In [8]:
  
def getDataFrameFromLicitacaoVencedor(datas):
    columns = ['DataReferencia', 'cdIBGE', 'dsFormaPagamento', 'dsItem',
       'dsModalidadeLicitacao', 'dsTipoEntregaProduto', 'dsUnidadeMedida',
       'dtHomologacao', 'dtPrazoEntregaPropostaLicitacao',
       'dtValidadeProposta', 'idPessoa', 'idTipoEntregaProduto',
       'idUnidadeMedida', 'idlicitacao', 'nmEntidade', 'nmMunicipio',
       'nmPessoa', 'nrAnoLicitacao', 'nrClassificacao', 'nrDocumento',
       'nrItem', 'nrLicitacao', 'nrLote', 'nrPrazoLimiteEntrega',
       'nrQuantidade', 'nrQuantidadePropostaLicitacao',
       'nrQuantidadeVencedorLicitacao', 'ultimoEnvioSIMAMNesteExercicio',
       'vlLicitacaoVencedorLicitacao', 'vlMaximoTotal', 'vlMaximoUnitarioitem',
       'vlPropostaItem', 'vlMinimoTotal', 'vlMinimoUnitarioItem']
    
    valuesToFill = {'vlMinimoTotal': 0,
                    'vlMinimoUnitarioItem': 0, 
                    'vlLicitacaoVencedorLicitacao': 0,
                    'nrQuantidadeVencedorLicitacao':0}
    df = pd.DataFrame(datas,columns=columns).fillna(value=valuesToFill)
    return df

def castingDataFrameLicitacaoVencedor(df_vencedor):
    casting = {
        'dtHomologacao' : 'datetime64[ns]',
        'dtPrazoEntregaPropostaLicitacao' : 'datetime64[ns]',
        'dtValidadeProposta' : 'datetime64[ns]',
        'nrClassificacao': 'int',
        'nrQuantidade' : 'float',
        'nrQuantidadePropostaLicitacao' : 'float',
        'nrQuantidadeVencedorLicitacao' : 'float',
        'vlUnitarioVencedorLicitacao' : 'float',
        'vlMaximoTotal' : 'float',
        'vlMaximoUnitarioitem' : 'float',
        'vlMinimoTotal' : 'float',
        'vlMinimoUnitarioItem' : 'float',
        'vlPropostaItem' : 'float'
    }
    df = df_vencedor.astype(casting,copy=False)
    return df

def processDataFrameLicitacaoVencedor (data):
    df = getDataFrameFromLicitacaoVencedor(data)
    df.rename(columns={'idPessoa':'idEntidade',
                       'nmPessoa': 'nmFornecedor',
                       'vlLicitacaoVencedorLicitacao':'vlUnitarioVencedorLicitacao'}, inplace=True)
    df = stripStringValues(df)
    df = castingDataFrameLicitacaoVencedor(df)
    df['vlTotalVencedorLicitacao'] = df['vlUnitarioVencedorLicitacao'].mul(df['nrQuantidadeVencedorLicitacao'], fill_value=0.0)
    return df

In [9]:
def getDataFrameFromLicitacaoParticipante(data):
    col = ['DataReferencia', 'cdIBGE', 'idLicitacao', 'idPessoa', 'nmEntidade',
       'nmMunicipio', 'nmParticipanteLicitacao', 'nrDocParticipanteLicitacao',
       'sgDocParticipanteLicitacao', 'ultimoEnvioSIMAMNesteExercicio']
    return pd.DataFrame(data, collumns=col)

def processDataFrameLicitacaoParticipante(data):
    df = getDataFrameFromLicitacao(data)
    df.rename(columns={'idPessoa': 'idEntidade'}, inplace=True)
    df = stripStringValues(df)
    return df

In [10]:
config_files_type = {
    'Licitacao.xml' : {
        'parser' : processDataFrameLicitacao,
        'collection' : 'licitacao'
    },
    'LicitacaoVencedor.xml' : {
        'parser' : processDataFrameLicitacaoVencedor,
        'collection' : 'licitacaoVencedor'
    },
    'LicitacaoParticipante.xml' : {
        'parser' : processDataFrameLicitacaoParticipante,
        'collection' : 'licitacaoParticipante'
    }
}

In [11]:
def handler_files(filenames):
    for file in filenames:
        file_type = determine_file_type(file)
        parser = config_files_type[file_type]['parser']
        collection_name = config_files_type[file_type]['collection']
        root = readXML(file)
        data = processRootElement(root)
        tam_data = len(data)
        df = parser(data)
        result = save_mongo(df, collection_name=collection_name)
        tam_result = len(result.inserted_ids)
        if(tam_data == tam_result):
            print('success: ', file)
        else :
            print('verificar: ', file)

In [12]:
handler_files(files)

success:  data/licitacao/2013_410010_Licitacao.xml
success:  data/licitacao/2013_410010_LicitacaoParticipante.xml
success:  data/licitacao/2013_410010_LicitacaoVencedor.xml
success:  data/licitacao/2013_410110_Licitacao.xml
success:  data/licitacao/2013_410110_LicitacaoParticipante.xml
success:  data/licitacao/2013_410110_LicitacaoVencedor.xml
success:  data/licitacao/2013_410270_Licitacao.xml
success:  data/licitacao/2013_410270_LicitacaoParticipante.xml
success:  data/licitacao/2013_410270_LicitacaoVencedor.xml
success:  data/licitacao/2013_410360_Licitacao.xml
success:  data/licitacao/2013_410360_LicitacaoParticipante.xml
success:  data/licitacao/2013_410360_LicitacaoVencedor.xml
success:  data/licitacao/2013_410640_Licitacao.xml
success:  data/licitacao/2013_410640_LicitacaoParticipante.xml
success:  data/licitacao/2013_410640_LicitacaoVencedor.xml
success:  data/licitacao/2013_410970_Licitacao.xml
success:  data/licitacao/2013_410970_LicitacaoParticipante.xml
success:  data/licita

success:  data/licitacao/2016_411370_Licitacao.xml
success:  data/licitacao/2016_411370_LicitacaoParticipante.xml
success:  data/licitacao/2016_411370_LicitacaoVencedor.xml
success:  data/licitacao/2016_411990_Licitacao.xml
success:  data/licitacao/2016_411990_LicitacaoParticipante.xml
success:  data/licitacao/2016_411990_LicitacaoVencedor.xml
success:  data/licitacao/2016_412070_Licitacao.xml
success:  data/licitacao/2016_412070_LicitacaoParticipante.xml
success:  data/licitacao/2016_412070_LicitacaoVencedor.xml
success:  data/licitacao/2016_412410_Licitacao.xml
success:  data/licitacao/2016_412410_LicitacaoParticipante.xml
success:  data/licitacao/2016_412410_LicitacaoVencedor.xml
success:  data/licitacao/2016_412660_Licitacao.xml
success:  data/licitacao/2016_412660_LicitacaoParticipante.xml
success:  data/licitacao/2016_412660_LicitacaoVencedor.xml
success:  data/licitacao/2016_412850_Licitacao.xml
success:  data/licitacao/2016_412850_LicitacaoParticipante.xml
success:  data/licita